# **FullControl Lampshade**

Click the ▶︎ button on the left of the cell below to connect to python running on Google's servers and generate a preview of the model

Change the sliders to edit the design, and click the ▶︎ button again to regenerate the preview. Change the `Output` option to `GCode` to download Gcode - make sure you've set your printer settings correctly below

If you're feeling adventurous, double click `show code` below the parameter sliders and then you can adjust anything you like about the design. It's a very advanced, mathematical design though - much simpler examples are provided in [FullControl tutorial notebooks](https://colab.research.google.com/github/FullControlXYZ/fullcontrol/blob/master/tutorials/colab/contents_colab.ipynb)

The entire geometry and manufacturing procedure are defined in the code below in <0.01 MB ... that's something like **0.001% of an stl file** that might be required for this model, which wouldn't even include any design details like printed-line sequence/direction/speed/width

In addition to code for the FullControl model design, the code below also entirely defines the UI of this page (e.g. sliders) - it's a great example of how a complex design can be shared with non-expert users

<<< check out other demo models [here](https://github.com/FullControlXYZ/fullcontrol/tree/master/models/README.md) >>>

<picture>
<img src="https://github.com/FullControlXYZ/media/blob/main/lampshade_collage.jpg?raw=true">
</picture>

In [ ]:
# @title Intro video - Colab walkthrough during 0m:58s to 1m:52s: { display-mode: "form"}
%%html
<div><iframe width="640" height="360" src=https://youtube.com/embed/8oDpTmUya6I></iframe></div>

In [ ]:
# VS Code / Jupyter interactive controls (kept close to the 3D preview)
import sys
from math import tau, sin, cos, exp, pi

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output, FileLink
except Exception as e:
    widgets = None
    _widgets_import_error = e


def _generate_lampshade(
    Output: str = 'Detailed Plot',
    Annotations: bool = True,
    Printer_name: str = 'generic',
    Nozzle_temp: int = 220,
    Bed_temp: int = 40,
    Fan_percent: int = 100,
    Material_flow_percent: int = 100,
    Print_speed_percent: int = 200,
    Design_name: str = 'fc_lampshade',
    Height: int = 150,
    Nominal_radius: int = 34,
    Tip_length: int = 20,
    Star_tips: int = 6,
    Main_bulge: float = 22.5,
    Secondary_bulges: float = 15,
    Secondary_bulge_count: int = 2,
    Twist_turns: float = 0.0,
    Inner_frame_hole_diameter: int = 30,
    Inner_frame_height: int = 3,
    Inner_frame_wave_amplitude: float = 17.5,
    Centre_XY: int = 104,
    # Advanced parameters (kept as-is from the Colab model)
    zag_min: float = 1,
    zag_max: float = 5,
    zigzag_freq_factor: float = 1.0,
    zigzag_radius_factor: float = 1.0,
    zigzag_rounding_radius: int = 0,
    rip_depth: float = 0.5,
    rip_freq: int = 30,
    swerve: float = 0.02,
    segs_shell: int = 300,
    x_1: float = 6,
    x_2: float = 30,
    frame_width_factor: float = 2.5,
    frame_line_spacing_ratio: float = 0.2,
    layer_ratio: int = 2,
    start_angle: float = 0.75 * tau,
    frame_overlap: float = 2.5,
    segs_frame: int = 64,
    EH: float = 0.2,
    EW: float = 0.5,
    initial_print_speed: int = 500,
    main_print_speed: int = 1500,
    speedchange_layers: int = 5,
    initial_z_factor: float = 0.7,
    viewer_point_stride: int = 1,
    viewer_layer_stride: int = 1,
    ):
    import fullcontrol as fc
    import lab.fullcontrol as fclab

    target = 'visualize' if Output in ['Detailed Plot', 'Simple Plot'] else 'gcode'

    height, r_0, tip_len, n_tip = Height, Nominal_radius, Tip_length, Star_tips
    bulge1, bulge2 = Main_bulge, Secondary_bulges
    bulge2_count = max(0, int(Secondary_bulge_count))
    frame_rad_inner = Inner_frame_hole_diameter / 2
    frame_height = Inner_frame_height
    amp_1 = Inner_frame_wave_amplitude
    centre_xy = Centre_XY

    frame_rad_max = r_0 + tip_len + frame_overlap
    frame_rad_inner += EW / 2  # allow for extrusion width

    if Output == 'Simple Plot':
        EH, segs_shell = EH * 30, max(1, n_tip * 20)
    elif Output == 'Detailed Plot':
        EH = EH * 10

    shell_layers = int(height / EH)
    frame_layers = int(frame_height / EH) if frame_height > 0 else 0
    initial_z = EH * initial_z_factor

    steps = []
    segs_shell_wave = int(segs_shell)
    # Zigzag round: 0 = sharp (triangle-like); 10 = smooth (circle-like).
    # We do two things:
    # 1) morph the zigzag waveform from triangle -> raised cosine
    # 2) increase sampling so the smooth curve can actually appear (otherwise it will still look triangular)
    _round_t = max(0.0, min(10.0, float(zigzag_rounding_radius))) / 10.0
    _zigzag_cycles = max(1e-6, (segs_shell_wave / 2.0) * float(zigzag_freq_factor))
    _desired_pts_per_cycle = 2.0 + (_round_t * 10.0)  # 2..12 points per zigzag around the perimeter
    segs_shell_samples = int(max(20.0, max(float(segs_shell), _zigzag_cycles * _desired_pts_per_cycle)))
    t_steps_shell = fc.linspace(0, 1, segs_shell_samples + 1)
    t_steps_frame_line = fc.linspace(0, 1, segs_frame + 1)

    for layer in range(shell_layers):
        if target == 'visualize' and viewer_layer_stride > 1 and layer != 0 and (layer % viewer_layer_stride != 0):
            continue
        if layer <= speedchange_layers:
            print_speed = initial_print_speed + (main_print_speed - initial_print_speed) * (layer / speedchange_layers)
        else:
            print_speed = main_print_speed
        z_now = initial_z + layer * EH
        z_fraction = z_now / height
        twist_angle = (tau * float(Twist_turns) * z_fraction) if Twist_turns else 0.0
        centre_now = fc.Point(x=centre_xy, y=centre_xy, z=z_now)
        shell_steps, wave_steps = [], []

        for t_now in t_steps_shell[: int((segs_shell_samples / max(n_tip, 1)) / 2) + 1]:
            a_now = start_angle + (tau * t_now)
            angular_swerve = -(
                (swerve * tau * sin(t_now * n_tip * tau + (tau / 2)))
                * (
                    (
                        (1 / (1 + exp(x_1 - z_fraction * x_2)))
                        * (1 / (1 + exp(x_1 - (1 - z_fraction) * x_2)))
                    )
                    - (0.5 * (sin(z_fraction * 0.5 * tau)) ** 20)
                )
            )
            star_shape_wave = tip_len * (0.5 + 0.5 * (cos(t_now * n_tip * tau))) ** 2.5
            primary_z_wave = bulge1 * (sin(z_fraction * 0.5 * tau)) ** 1
            if bulge2_count <= 0:
                secondary_z_waves = 0
            else:
                # bulge2_count is the number of secondary bulges over the height (roughly).
                secondary_z_waves = bulge2 * (0.5 + 0.5 * (cos((z_fraction + 0.15) * float(bulge2_count) * tau))) ** 1.5

            # Zigzag waveform morph: triangle (sharp) -> raised cosine (smooth)
            _phase = t_now * _zigzag_cycles
            _frac = _phase - int(_phase)  # safe floor because _phase >= 0
            _tri = 1.0 - abs((2.0 * _frac) - 1.0)  # 0..1 triangle wave
            _rcos = 0.5 - (0.5 * cos(2.0 * pi * _phase))  # 0..1 raised cosine
            zigzag_base = ((1.0 - _round_t) * _tri) + (_round_t * _rcos)

            zigzag_depth = zag_min + (zag_max * (0.5 + 0.5 * (cos(t_now * n_tip * tau))) ** 2)
            zigzag_wave = (zigzag_base * zigzag_depth) * zigzag_radius_factor if Output != 'Simple Plot' else 0
            tiny_z_ripples = (rip_depth * (sin(z_fraction * rip_freq * tau)) ** 2) if Output != 'Simple Plot' else 0
            r_now = r_0 + star_shape_wave + primary_z_wave + secondary_z_waves + zigzag_wave + tiny_z_ripples
            shell_steps.append(fc.polar_to_point(centre_now, r_now, a_now + angular_swerve))

        shell_steps.extend(fclab.reflectXYpolar_list(shell_steps, centre_now, start_angle + pi / max(n_tip, 1)))
        shell_steps = fc.move_polar(shell_steps, centre_now, 0, tau / max(n_tip, 1), copy=True, copy_quantity=n_tip)
        if Twist_turns:
            shell_steps = fc.move_polar(shell_steps, centre_now, 0, twist_angle)
        if target == 'visualize' and viewer_point_stride > 1:
            _eff_stride = int(viewer_point_stride)
            # Avoid downsampling so hard that zigzags disappear (aliasing), especially when rounding is high.
            if Output != 'Simple Plot' and zigzag_radius_factor > 0 and (zag_min > 0 or zag_max > 0):
                _points_per_cycle = float(segs_shell_samples) / max(_zigzag_cycles, 1e-6)
                _min_pts_per_cycle = 3.0 + (_round_t * 9.0)  # 3..12
                _max_stride = int(_points_per_cycle // _min_pts_per_cycle)
                _eff_stride = max(1, min(_eff_stride, max(1, _max_stride)))
            shell_steps = shell_steps[:: _eff_stride]
            if len(shell_steps) > 0:
                shell_steps.append(shell_steps[0])
        steps.extend([fc.ExtrusionGeometry(width=EW, height=EH), fc.Printer(print_speed=print_speed)] + shell_steps)

        if (
            (target == 'gcode' and layer % layer_ratio == layer_ratio - 1 and layer < frame_layers)
            or (target == 'visualize' and layer == 0 and frame_height > 0)
        ):
            for t_now in t_steps_frame_line:
                x_now = centre_xy + (frame_line_spacing_ratio * (frame_width_factor * EW)) + (amp_1 * t_now) * (
                    (0.5 - 0.5 * cos((t_now ** 0.66) * 3 * tau)) ** 1
                )
                y_now = centre_xy - frame_rad_inner - ((frame_rad_max - frame_rad_inner) * (1 - t_now))
                wave_steps.append(fc.Point(x=x_now, y=y_now, z=z_now))
            wave_steps.extend(fc.arcXY(centre_now, frame_rad_inner, start_angle, pi / max(n_tip, 1), int(64 / max(n_tip, 1))))
            wave_steps.extend(fclab.reflectXYpolar_list(wave_steps, centre_now, start_angle + pi / max(n_tip, 1)))
            wave_steps = fc.move_polar(wave_steps, centre_now, 0, tau / max(n_tip, 1), copy=True, copy_quantity=n_tip)
            if Twist_turns:
                wave_steps = fc.move_polar(wave_steps, centre_now, 0, twist_angle)
            steps.append(fc.ExtrusionGeometry(width=EW * frame_width_factor, height=EH * layer_ratio))
            steps.append(fc.Printer(print_speed=print_speed / (frame_width_factor * layer_ratio)))
            steps.extend(wave_steps)

    if Output == 'Simple Plot':
        steps.append(fc.PlotAnnotation(point=fc.Point(x=centre_xy, y=50, z=0), label='Not all layers previewed - nor ripple texture'))
    if Output == 'Detailed Plot':
        steps.append(fc.PlotAnnotation(point=fc.Point(x=centre_xy, y=50, z=0), label='Not all layers previewed'))
    steps.append(fc.PlotAnnotation(point=fc.Point(x=centre_xy, y=25, z=0), label=f'Speed increases from {initial_print_speed} to {main_print_speed} mm/min during first {speedchange_layers} layers'))
    steps.append(fc.PlotAnnotation(point=fc.Point(x=centre_xy, y=0, z=0), label='Avoid larger overhangs than default design - ripple texture exacerbates overhangs'))
    steps.append(fc.PlotAnnotation(point=fc.Point(x=centre_xy, y=centre_xy, z=height + 10), label='Try doubling speed - you may need to increase nozzle temperature'))

    gcode_controls = fc.GcodeControls(
        printer_name=Printer_name,
        save_as=Design_name,
        initialization_data={
            'primer': 'front_lines_then_y',
            'print_speed': initial_print_speed,
            'nozzle_temp': Nozzle_temp,
            'bed_temp': Bed_temp,
            'fan_percent': Fan_percent,
            'material_flow_percent': Material_flow_percent,
            'print_speed_percent': Print_speed_percent,
            'extrusion_width': EW,
            'extrusion_height': EH,
        },
    )
    plot_controls = fc.PlotControls(style='line', zoom=0.6, initialization_data={'extrusion_width': EW, 'extrusion_height': EH})
    plot_controls.hide_annotations = not Annotations

    if target == 'gcode':
        gcode = fc.transform(steps, 'gcode', gcode_controls)
        gcode_path = f'{Design_name}.gcode'
        with open(gcode_path, 'w', encoding='utf-8') as f:
            f.write(gcode)
        try:
            from google.colab import files  # type: ignore
            files.download(gcode_path)
        except Exception:
            try:
                display(FileLink(gcode_path))
            except Exception:
                print(f'Wrote: {gcode_path}')
        return gcode_path
    else:
        fc.transform(steps, 'plot', plot_controls)
        return None


# If running in Colab, the next cell's sliders already provide controls.
_in_colab = 'google.colab' in sys.modules
if _in_colab:
    print('Colab detected: use the sliders in the next cell for controls.')
elif widgets is None:
    print('ipywidgets is not available in this environment. Install/enable ipywidgets to use interactive controls.')
    print(f'Import error: {_widgets_import_error!r}')
else:
    output_widget = widgets.Dropdown(options=['Simple Plot', 'Detailed Plot', 'GCode'], value='Detailed Plot', description='Output')
    viewer_widget = widgets.Dropdown(options=['Fast viewer', 'Normal viewer', 'High detail'], value='Normal viewer', description='Viewer')
    annotations_widget = widgets.Checkbox(value=True, description='Annotations')

    printer_widget = widgets.Dropdown(
        options=['generic', 'ultimaker2plus', 'prusa_i3', 'ender_3', 'cr_10', 'bambulab_x1', 'toolchanger_T'],
        value='generic',
        description='Printer',
    )
    nozzle_widget = widgets.IntText(value=220, description='Nozzle °C')
    bed_widget = widgets.IntText(value=40, description='Bed °C')
    fan_widget = widgets.IntText(value=100, description='Fan %')
    flow_widget = widgets.IntText(value=100, description='Flow %')
    speed_widget = widgets.IntText(value=200, description='Speed %')
    name_widget = widgets.Text(value='fc_lampshade', description='Name')

    height_widget = widgets.IntSlider(value=150, min=100, max=200, step=10, description='Height')
    radius_widget = widgets.IntSlider(value=34, min=20, max=50, step=1, description='Radius')
    tip_widget = widgets.IntSlider(value=20, min=10, max=30, step=2, description='Tip len')
    stars_widget = widgets.IntSlider(value=6, min=0, max=8, step=1, description='Star tips')
    bulge1_widget = widgets.FloatSlider(value=22.5, min=0, max=25, step=2.5, description='Main bulge')
    bulge2_widget = widgets.FloatSlider(value=15, min=0, max=20, step=2.5, description='2nd bulge')
    bulge2_count_widget = widgets.IntSlider(value=2, min=0, max=6, step=1, description='Sec bulges')
    twist_widget = widgets.FloatSlider(value=0.0, min=-2.0, max=2.0, step=0.05, description='Twist')
    frame_diam_widget = widgets.BoundedIntText(value=30, min=0, max=200, step=1, description='Frame hole')
    frame_h_widget = widgets.IntSlider(value=3, min=0, max=10, step=1, description='Frame ht')
    frame_amp_widget = widgets.FloatText(value=17.5, description='Frame amp')
    centre_widget = widgets.IntText(value=104, description='Centre XY')

    zigzag_min_widget = widgets.FloatSlider(value=1.0, min=0.0, max=6.0, step=0.25, description='Zigzag min')
    zigzag_max_widget = widgets.FloatSlider(value=5.0, min=0.0, max=10.0, step=0.25, description='Zigzag max')
    zigzag_freq_widget = widgets.FloatSlider(value=1.0, min=0.25, max=3.0, step=0.05, description='Zigzag freq')
    zigzag_radius_widget = widgets.FloatSlider(value=1.0, min=0.0, max=3.0, step=0.05, description='Zigzag radius')
    zigzag_round_widget = widgets.IntSlider(value=0, min=0, max=10, step=1, description='Zigzag round')

    run_button = widgets.Button(description='Generate / Update')
    plot_output = widgets.Output()

    controls_box = widgets.VBox(
        [
            widgets.HTML('<b>Controls (VS Code / Jupyter)</b>'),
            output_widget,
            viewer_widget,
            annotations_widget,
            widgets.HTML('<b>Printer parameters</b>'),
            printer_widget,
            nozzle_widget,
            bed_widget,
            fan_widget,
            flow_widget,
            speed_widget,
            name_widget,
            widgets.HTML('<b>Design parameters</b>'),
            height_widget,
            radius_widget,
            tip_widget,
            stars_widget,
            bulge1_widget,
            bulge2_widget,
            bulge2_count_widget,
            twist_widget,
            frame_diam_widget,
            frame_h_widget,
            frame_amp_widget,
            centre_widget,
            widgets.HTML('<b>Line / Zigzag controls</b>'),
            zigzag_min_widget,
            zigzag_max_widget,
            zigzag_freq_widget,
            zigzag_radius_widget,
            zigzag_round_widget,
            run_button,
        ]
    )

    def _on_run_clicked(_):
        with plot_output:
            clear_output(wait=True)
            if viewer_widget.value == 'Fast viewer':
                _viewer_point_stride, _viewer_layer_stride = 6, 4
            elif viewer_widget.value == 'High detail':
                _viewer_point_stride, _viewer_layer_stride = 1, 1
            else:
                _viewer_point_stride, _viewer_layer_stride = 2, 2
            _generate_lampshade(
                Output=output_widget.value,
                Annotations=annotations_widget.value,
                Printer_name=printer_widget.value,
                Nozzle_temp=nozzle_widget.value,
                Bed_temp=bed_widget.value,
                Fan_percent=fan_widget.value,
                Material_flow_percent=flow_widget.value,
                Print_speed_percent=speed_widget.value,
                Design_name=name_widget.value,
                Height=height_widget.value,
                Nominal_radius=radius_widget.value,
                Tip_length=tip_widget.value,
                Star_tips=stars_widget.value,
                Main_bulge=bulge1_widget.value,
                Secondary_bulges=bulge2_widget.value,
                Secondary_bulge_count=bulge2_count_widget.value,
                Twist_turns=twist_widget.value,
                Inner_frame_hole_diameter=frame_diam_widget.value,
                Inner_frame_height=frame_h_widget.value,
                Inner_frame_wave_amplitude=frame_amp_widget.value,
                Centre_XY=centre_widget.value,
                zag_min=zigzag_min_widget.value,
                zag_max=zigzag_max_widget.value,
                zigzag_freq_factor=zigzag_freq_widget.value,
                zigzag_radius_factor=zigzag_radius_widget.value,
                zigzag_rounding_radius=zigzag_round_widget.value,
                viewer_point_stride=_viewer_point_stride,
                viewer_layer_stride=_viewer_layer_stride,
            )

    run_button.on_click(_on_run_clicked)
    display(controls_box)
    display(plot_output)
    _on_run_clicked(None)

Output()

In [21]:
# @title **Lampshade Design** { display-mode: "form"}

# @markdown #### **🡸 Click ▶︎ button to connect to python** --- *connection may take ~20 seconds*
# @markdown  #### **🡸 Click it again** to regenerate the design after changing parameters **(or press *`shift + enter`)*** --- *gcode generation may take ~20 seconds*

# import python packages (if not already imported)
import fullcontrol as fc
import lab.fullcontrol as fclab
from math import tau, sin, cos, exp, pi

# output type (widget)
# @markdown ---
# @markdown ### **Controls:**
Output = 'Detailed Plot' # @param ["Simple Plot", "Detailed Plot", "GCode"]
target = 'visualize' if Output in ['Detailed Plot', 'Simple Plot'] else 'gcode' # 'visualize' or 'gcode'
Annotations = True # @param {type:"boolean"}


# setup printer parameters
# @markdown ### **Printer parameters:**
Printer_name = 'generic' # @param ['generic', 'ultimaker2plus', 'prusa_i3',  'ender_3', 'cr_10', 'bambulab_x1', 'toolchanger_T']
Nozzle_temp = 220 # @param {type:"number"} # default 220
Bed_temp = 40 # @param {type:"number"} # default 40
Fan_percent = 100 # @param {type:"number"} # default 100
Material_flow_percent = 100 # @param {type:"number"} # default 100
Print_speed_percent = 200 # @param {type:"number"} # default 100
Design_name = 'fc_lampshade'

# design parameters (widgets)
# @markdown ### **Design parameters:**
Height = 150 # @param {type:"slider", min:100, max:200, step:10}
# default 150 # Overall height of the lampshade
Nominal_radius = 34 # @param {type:"slider", min:20, max:50, step:1}
# default 34 # Initial radius of teh shell before all the modifications like bulges
Tip_length = 20 # @param {type:"slider", min:10, max:30, step:2}
# default 20 # the extra radius to add to the initial radius for the pointy bits
Star_tips = 6 # @param {type:"slider", min:0, max:8, step:1}
# default 6 # Number of pointy star tips
Main_bulge = 22.5 # @param {type:"slider", min:0, max:25, step:2.5}
# default 22.5 # increase in radius to get an overall bulbous shape (wider half-way up than at top/bottom.
Secondary_bulges = 15 # @param {type:"slider", min:0, max:20, step:2.5}
# default 15 # increase in radius for the two bulges about 1/3 and 2.3 of the way up the overall height of the lampshade
Secondary_bulge_count = 2 # @param {type:"slider", min:0, max:6, step:1}
# default 2 # Number of secondary bulges along the height (0 disables them)
Twist_turns = 0.0 # @param {type:"slider", min:-2.0, max:2.0, step:0.05}
# default 0.0 # Twist (turns) from bottom to top (rotates each layer)
Inner_frame_hole_diameter = 30 # @param {type:"number"}
# default 30 # Diameter of the inner ring of the inner frame - this is increased autoamtcially to allow for extrusion width
Inner_frame_height = 3 # @param {type:"slider", min:0, max:10, step:1}
# default 3 # Height the inner frame is printed
Inner_frame_wave_amplitude = 17.5 # @param {type:"number"}
# default 17.5 # This is the amplitude of the wave lines of the inner frame. To get neighbouring wavey lines to touch, this will need to be adjusted if the lengths of the lines changes.
Centre_XY = 104 # @param {type:"number"}
# default 105 # Centre of the lampshade on the print bed in XY
# automatically convert widget design parameters to variables used in the python design
height, r_0, tip_len, n_tip, bulge1, bulge2, frame_rad_inner, frame_height, amp_1, centre_xy = Height, Nominal_radius, Tip_length, Star_tips, Main_bulge, Secondary_bulges, Inner_frame_hole_diameter/2, Inner_frame_height, Inner_frame_wave_amplitude, Centre_XY

# advanced design parameters
# shell parameters:
zag_min, zag_max = 1, 5 # Depth of zigzags at the flat region of the shade zag_min (where they don't need to be deep to achieve a nice rippled texture because they are closer together (smaller radius)) and at the star tips zag_max (where they need to be deeper to still look look good deep ridges)
rip_depth, rip_freq = 0.5, 30 # Depth of tiny ripples in the Z direction and how many tiny ripples there are in the Z direction over the whole height
swerve = 0.02 # radians #  The swerving streamlines are achieved by angular offsets. This number controls the magnitude of those offsets and the amount of swerve.
segs_shell = 300 # Segments in the whole perimeter of the shell
x_1, x_2 = 6, 30 # These parameters controls how high up the shade the streamline swerves come into effect (and return to original shape at the top of the lampshade)
# inner-frame parameters:
frame_width_factor = 2.5 # The extrusion width of the inner frame is greater than the extrusion width of the outer shade by theis factor
frame_line_spacing_ratio = 0.2 # How laterally offset are the lines of the inner frame when they kiss (as a fraction of extrusion width) - the centre to centre distance of the lines is 2 * this value * frame line extrusion width
layer_ratio = 2 # this number is the number of layers of the shell that are printed for every layer of the frame
start_angle = 0.75*tau # this cannot be modified without also modifying the direction of the first frame line (currently in +Y direction)
frame_overlap =  2.5 # how much the inner frame overlaps with the shell
segs_frame = 64 # Segments per single wavey line in the inner frame
# printing parameters:
EH, EW = 0.2, 0.5 # Extrusion height and width for the shell of the lamp shade
initial_print_speed, main_print_speed, speedchange_layers  = 500, 1500, 5 # print speed ramps up from initial_print_speed to main_print_speed over the first few layers (speedchange_layers)


# automatically calculated parameters
frame_rad_max = r_0+tip_len+frame_overlap # Outer radius of the wavey lines of the inner frame - this value is set so the lines connect will with the outer shade section, but don't cross it too much
# centre_xy = frame_rad_max+bulge1+bulge2+tip_len-10
frame_rad_inner += EW/2 # increase the hole size to allow for the extrusion width
if Output == 'Simple Plot': EH, segs_shell = EH*30, n_tip*20
elif Output == 'Detailed Plot': EH = EH*10
total_segs = segs_shell*(height/EH)
shell_layers, frame_layers = int(height/EH), int(frame_height/EH)
initial_z = EH*0.7 # initial nozzle position is set lower than the extrusion height to get a bit of 'squish' for good bed adhesion

# generate design
steps = []
t_steps_shell, t_steps_frame_line = fc.linspace(0, 1, segs_shell+1), fc.linspace(0, 1, segs_frame+1)
for layer in range(shell_layers):
    if layer <= speedchange_layers: print_speed = initial_print_speed + (main_print_speed-initial_print_speed)*(layer/speedchange_layers)
    z_now = initial_z + layer*EH
    z_fraction = z_now/height
    centre_now = fc.Point(x=centre_xy, y=centre_xy, z=z_now)
    shell_steps, wave_steps = [], []
    for t_now in t_steps_shell[:int((segs_shell/n_tip)/2)+1]: # calculate point in the first half of the region between the first and second 'star tips'
        a_now = start_angle+(tau*t_now) # increase polar angle for each point
        angular_swerve = -((swerve*tau*sin(t_now*n_tip*tau+(tau/2))))*(((1/(1+exp(x_1-z_fraction*x_2)))*(1/(1+exp(x_1-(1-z_fraction)*x_2))))-(0.5*(sin(z_fraction*0.5*tau))**20)) # modify the polar angle of the current point to achieve a swerving shape in an opposing symmetric clockwise/anti-clockwise manner to make bulges appear to grow wider
        star_shape_wave = (tip_len*(0.5+0.5*(cos(t_now*n_tip*tau)))**2.5) # radial modifier for the current point due to the start-tip shape
        primary_z_wave = (bulge1*(sin(z_fraction*0.5*tau))**1) # radial modifier for the current point due to the main bulge
        secondary_z_waves = (((bulge2*(0.5+0.5*(cos((z_fraction+0.15)*Secondary_bulge_count*tau)))**1.5))) if Secondary_bulge_count > 0 else 0 # radial modifier for the current point due to the secondary bulges
        zigzag_wave = ((0.5-(0.5*cos(t_now*(segs_shell/2)*2*pi)))*(zag_min+(zag_max*(0.5+0.5*(cos(t_now*n_tip*tau)))**2))) if Output != 'Simple Plot' else 0 # radial modifier for the current point due to the zig-zag pattern around the circumference
        tiny_z_ripples = (rip_depth*(sin(z_fraction*rip_freq*tau))**2) if Output != 'Simple Plot' else 0 # radial modifier for the current point due to the tiny waves in the z direction
        r_now = r_0 + star_shape_wave + primary_z_wave + secondary_z_waves + zigzag_wave + tiny_z_ripples
        shell_steps.append(fc.polar_to_point(centre_now, r_now, a_now+angular_swerve)) # add the point to the current list of points based on the current angle and radius
    shell_steps.extend(fclab.reflectXYpolar_list(shell_steps, centre_now, start_angle+pi/n_tip)) # create the second half of the region between the first and second 'star tips' to achieve a rotationally repeating unit cell
    shell_steps = fc.move_polar(shell_steps, centre_now, 0, tau/n_tip, copy=True, copy_quantity=n_tip) # repeat the unit cell for all 'start tips'
    if Twist_turns != 0:
        shell_steps = fc.move_polar(shell_steps, centre_now, 0, tau*Twist_turns*z_fraction)
    # previous_layer_point = shell_steps[-1]
    # if layer >0: print(f'approximate overhang for layer {layer}: {(360/tau)*fclab.angleZ(steps[-1], previous_layer_point)} degrees (exluding overhang due to mini z ripples)')
    steps.extend([fc.ExtrusionGeometry(width=EW, height=EH), fc.Printer(print_speed=print_speed)] + shell_steps) # set the current speed and extrusion geometry and add all the points for the shell for this layer
    if (target == 'gcode' and layer % layer_ratio == layer_ratio-1 and layer < frame_layers) or (target == 'visualize' and layer == 0 and frame_height > 0): # only print every few layers for gcode or the first layer for visualizing
        for t_now in t_steps_frame_line: # generate a wavey line in the Y direction - this assumes "start_angle = 0.75*tau"
            x_now = centre_xy+(frame_line_spacing_ratio*(frame_width_factor*EW))+(amp_1*t_now)*((0.5-0.5*cos((t_now**0.66)*3*tau))**1)
            y_now = centre_xy - frame_rad_inner - ((frame_rad_max-frame_rad_inner)*(1-t_now))
            wave_steps.append(fc.Point(x=x_now, y=y_now, z=z_now))
        wave_steps.extend(fc.arcXY(centre_now, frame_rad_inner, start_angle, pi/n_tip, int(64/n_tip))) # add points for half of the arc between the first and second 'star tips'
        wave_steps.extend(fclab.reflectXYpolar_list(wave_steps, centre_now, start_angle+pi/n_tip)) # reflect the arc and wavey line to complete one rotationally repreating unit cell
        wave_steps = fc.move_polar(wave_steps, centre_now, 0, tau/n_tip, copy=True, copy_quantity=n_tip) # repeat the unit cell for each 'star tip'
        if Twist_turns != 0:
            wave_steps = fc.move_polar(wave_steps, centre_now, 0, tau*Twist_turns*z_fraction)
        steps.append(fc.ExtrusionGeometry(width=EW*frame_width_factor, height=EH*layer_ratio))
        steps.append(fc.Printer(print_speed=print_speed/(frame_width_factor*layer_ratio)))
        steps.extend(wave_steps)

# add annotations
if Output == 'Simple Plot': steps.append(fc.PlotAnnotation(point=fc.Point(x=centre_xy, y=50, z=0), label='Not all layers previewed - nor ripple texture'))
if Output == 'Detailed Plot': steps.append(fc.PlotAnnotation(point=fc.Point(x=centre_xy, y=50, z=0), label='Not all layers previewed'))
steps.append(fc.PlotAnnotation(point=fc.Point(x=centre_xy, y=25, z=0), label=f'Speed increases from {initial_print_speed} to {main_print_speed} mm/min during first {speedchange_layers} layers'))
steps.append(fc.PlotAnnotation(point=fc.Point(x=centre_xy, y=0, z=0), label=f'Avoid larger overhangs than default design - ripple texture exacerbates overhangs'))
steps.append(fc.PlotAnnotation(point=fc.Point(x=centre_xy, y=centre_xy, z=height+10), label=f'Try doubling speed - you may need to increase nozzle temperature'))

# transform design into plot or gcode
gcode_controls = fc.GcodeControls(printer_name=Printer_name, save_as = Design_name, initialization_data={'primer': 'front_lines_then_y', 'print_speed': initial_print_speed, 'nozzle_temp': Nozzle_temp, 'bed_temp': Bed_temp, 'fan_percent': Fan_percent, 'material_flow_percent': Material_flow_percent, 'print_speed_percent':Print_speed_percent, 'extrusion_width': EW, 'extrusion_height': EH})
plot_controls = fc.PlotControls(style='line', zoom=0.6, initialization_data={'extrusion_width': EW, 'extrusion_height': EH})
plot_controls.hide_annotations = False if Annotations else True
if target == 'gcode':
    gcode = fc.transform(steps, 'gcode', gcode_controls)
    open(f'{Design_name}.gcode', 'w').write(gcode)
    files.download(f'{Design_name}.gcode')
else:
    fc.transform(steps, 'plot', plot_controls)

# @markdown ##### **Details about the model parameters are given in code comments - many other parameters can also be edited in the code** *(double click this area to hide/show code)*